In [1]:
import os
import numpy as np
import aocd

In [2]:
current_day = 5
current_year = 2023
puzzle = aocd.models.Puzzle(year=current_year, day=current_day)
puzzle

<Puzzle(2023, 5) at 0x7fbef89b0400 - If You Give A Seed A Fertilizer>

# Part A

### Test data

In [19]:
test_data = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

In [25]:
def get_map(data):
    name, *maps = data.split('\n')
    # print(name, maps)
    property_map = {}
    for map_line in maps:
        dest, src, range_len = map(int, map_line.split())
        property_map[(src,src+range_len)] = (dest, dest+range_len) 
    return name, property_map

In [26]:
seeds, *maps = test_data.split('\n\n')
property_maps = {}
for map_line in maps:
    name, property_map = get_map(map_line)
    property_maps[name] = property_map
seeds = list(map(int,seeds.split()[1:]))
locations = []
for ip in ips:
    x = ip
    for map_name, property_map in property_maps.items():
        for (src_start, src_end), (dest_start, dest_end) in property_map.items():
            if src_start <= x < src_end:
                x = dest_start + (x - src_start)
                break
    locations.append(x)
    print(ip, x)
result = min(locations)
print(f"Result for test data = {result}")


79 82
14 43
55 86
13 35
Result for test data = 35


### Input data

In [27]:
ips, *maps = puzzle.input_data.split('\n\n')
property_maps = {}
for map_line in maps:
    name, property_map = get_map(map_line)
    property_maps[name] = property_map
ips = list(map(int,seeds.split()[1:]))
locations = []
for seed in seeds:
    x = seed
    for map_name, property_map in property_maps.items():
        for (src_start, src_end), (dest_start, dest_end) in property_map.items():
            if src_start <= x < src_end:
                x = dest_start + (x - src_start)
                break
    locations.append(x)
    print(seed, x)
result = min(locations)
print(f"Result for input data = {result}")


515785082 4202284817
87905039 2462188886
2104518691 4287508490
503149843 4050220846
720333403 2625685932
385234193 3507348822
1357904101 4074800133
283386167 892121398
93533455 2467817302
128569683 1104301931
2844655470 3337138858
24994629 1860608055
3934515023 2980390369
67327818 2441611665
2655687716 2734303452
8403417 457535844
3120497449 599623118
107756881 1083489129
4055128129 2353465194
9498708 458631135
Result for input data = 457535844


In [28]:
puzzle.answer_a = result

That's the right answer!  You are one gold star closer to restoring snow operations. You got rank 746 on this star's leaderboard. [Continue to Part Two]


# Part B

### Test data

In [84]:
import queue

def map_input_ranges_to_output_ranges(input_ranges_queue, property_map):
    output_ranges_queue = queue.Queue()
    while not input_ranges_queue.empty():
        # pop from queue
        start, end = input_ranges_queue.get()
        found_mapping = False
        for (src_start, src_end), (dest_start, dest_end) in property_map.items():
            # find intersection between the interval (start, end) and (src_start, src_end)
            # if the intersection is non-empty, then we have a match
            # print(start, end, src_start, src_end)
            start_in_src_range = src_start <= start < src_end
            end_in_src_range = src_start < end <= src_end
            if start_in_src_range and end_in_src_range: # entire range fits within
                # print(f"Original range ({start}, {end}) ", end="")
                start = dest_start + (start - src_start)
                end = dest_start + (end - src_start)
                # print(f"mapped to ({start},{end})")
                found_mapping = True
            elif start_in_src_range: # start is in the range, but end is not
                # map the part that is in the range, and create a new section and push to queue
                section1, section2 = (start, src_end), (src_end, end)
                start, end = section1
                input_ranges_queue.put(section2)
                start = dest_start + (start - src_start)
                end = dest_start + (end - src_start) # same as dest_end
                found_mapping = True
            elif end_in_src_range: # end is in the range, but start is not
                # map the part that is in the range, and create a new section and push to queue
                section1, section2 = (start, src_start), (src_start, end)
                start, end = section2
                input_ranges_queue.put(section1)
                start = dest_start + (start - src_start) # same as dest_start
                end = dest_start + (end - src_start)
                found_mapping = True
            if found_mapping:
                output_ranges_queue.put((start, end))
                break
        if not found_mapping:
            output_ranges_queue.put((start, end))
    # print(list(output_ranges_queue.queue))
    return output_ranges_queue

In [88]:
seeds, *maps = test_data.split('\n\n')
property_maps = {}
for map_line in maps:
    name, property_map = get_map(map_line)
    property_maps[name] = property_map
seeds = list(map(int,seeds.split()[1:]))
# print(property_maps)
# create of queue of ranges to process
ranges_queue = queue.Queue()
for start, range_len in zip(seeds[0::2], seeds[1::2]):
    ranges_queue.put((start, start+range_len))
    
# process the queue
for map_name, property_map in property_maps.items():
    print(map_name)
    ranges_queue = map_input_ranges_to_output_ranges(ranges_queue, property_map)
    print(list(ranges_queue.queue))

result = min(start for start, end in ranges_queue.queue)

print(f"Result for test data = {result}")

seed-to-soil map:
[(81, 95), (57, 70)]
soil-to-fertilizer map:
[(81, 95), (57, 70)]
fertilizer-to-water map:
[(81, 95), (53, 57), (61, 70)]
water-to-light map:
[(74, 88), (46, 50), (54, 63)]
light-to-temperature map:
[(45, 56), (82, 86), (90, 99), (78, 81)]
temperature-to-humidity map:
[(46, 57), (82, 86), (90, 99), (78, 81)]
humidity-to-location map:
[(60, 61), (86, 90), (94, 97), (82, 85), (46, 56), (56, 60), (97, 99)]
Result for test data = 46


### Input data

In [89]:
seeds, *maps = puzzle.input_data.split('\n\n')
property_maps = {}
for map_line in maps:
    name, property_map = get_map(map_line)
    property_maps[name] = property_map
seeds = list(map(int,seeds.split()[1:]))
# print(property_maps)
# create of queue of ranges to process
ranges_queue = queue.Queue()
for start, range_len in zip(seeds[0::2], seeds[1::2]):
    ranges_queue.put((start, start+range_len))
    
# process the queue
for map_name, property_map in property_maps.items():
    print(map_name)
    ranges_queue = map_input_ranges_to_output_ranges(ranges_queue, property_map)
    print(list(ranges_queue.queue))

result = min(start for start, end in ranges_queue.queue)

print(f"Result for input data = {result}")

seed-to-soil map:
[(1569194631, 1645303680), (1949944319, 1973031710), (41165510, 381174930), (1205326488, 1299264281), (1031035759, 1159605442), (2788621379, 2813616008), (2467976956, 2487930716), (2600617570, 2609020987), (2047051701, 2154808582), (2974155491, 2983654199), (381174930, 390316067), (1973031710, 2025334497), (1778186494, 1823411267), (1443025047, 1543924548), (4170255211, 4173483527), (1700053869, 1702708722), (3780457892, 4067445375), (969753308, 969898963), (2284917389, 2329063131), (3247991211, 3324095721), (634816620, 708275618), (2487930716, 2552598388), (619872400, 634816620)]
soil-to-fertilizer map:
[(3100287781, 3176396830), (2351738903, 2374826294), (202599792, 202945096), (1757456148, 1851393941), (421834722, 456352065), (304617695, 329612324), (3599604749, 3619558509), (3459362949, 3467766366), (2448846285, 2503601566), (1273867022, 1283365730), (2617666875, 2626808012), (2374826294, 2427129081), (1596284520, 1641509293), (3064134658, 3075017698), (3829585870

In [90]:
puzzle.answer_b = result

That's the right answer!  You are one gold star closer to restoring snow operations. You got rank 772 on this star's leaderboard.You have completed Day 5! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
